In [ ]:
import pandas as pd
import sys
sys.path.append('../src')
from table_generator import table_generator

# Create Tables with Block-wise Information on the Pyschopathology Symptom Scores

### Youth subset

In [ ]:
table = table_generator(SUBJID_filepath='../results/youth_SUBJIDs', membership_list_filepath='../results/youth_multiplex_Pearson_memberships.csv').table
table.to_csv('../results/youth_symptom_table.csv', index=False)
table

### Early adult subset

In [ ]:
table = table_generator(SUBJID_filepath='../results/early_adult_SUBJIDs', membership_list_filepath='../results/early_adult_multiplex_Pearson_memberships.csv').table
table.to_csv('../results/early_adult_symptom_table.csv', index=False)

# Permutation Tests for Between-block Comparisons of Mean SIP Scores

In [ ]:
# Create a dataframe of all subjects in the final sample
phenotype_df = pd.read_csv('[Insert filepath to phenotype data]', sep='\t', low_memory=False)
phenotype_df = phenotype_df[phenotype_df['INT_NUM'] == 1]
SUBJID_df = pd.concat([pd.read_csv('../results/youth_SUBJIDs'), pd.read_csv('../results/early_adult_SUBJIDs')], axis=0)
phenotype_df = pd.merge(phenotype_df, SUBJID_df['SUBJID'], how='inner', on='SUBJID')

In [ ]:
# define lists for the different categories of variables
ID_vars = ['SUBJID', 'INT_NUM', 'INT_TYPE', 'Sex']
SIP = ['SIP003', 'SIP004', 'SIP005', 'SIP006', 'SIP007', 'SIP008', 'SIP009', 'SIP010', 'SIP011', 'SIP012', 'SIP013', 'SIP014']

phenotype_df['SIP'] = phenotype_df[SIP].mean(axis=1)

In [ ]:
# Split the dataframe into the youth and early adult age groups
youth_phenotype_df = phenotype_df[phenotype_df['INT_TYPE'] == 'MP']
early_adult_phenotype_df = phenotype_df[phenotype_df['INT_TYPE'] == 'AP']

In [ ]:
# Create DataFrames with block membership columns for both the youth and early adult age groups

youth_membership_list_filepath = '../results/youth_multiplex_Pearson_memberships.csv'

youth_membership_list = pd.read_csv(youth_membership_list_filepath)['memberships'].values.tolist()
        
# save the number of blocks
Q_youth = max(youth_membership_list)
        
# add the block memberships to the phenotype dataframe
youth_phenotype_df['Block'] = youth_membership_list


early_adult_membership_list_filepath = '../results/early_adult_multiplex_Pearson_memberships.csv'

early_adult_membership_list = pd.read_csv(early_adult_membership_list_filepath)['memberships'].values.tolist()
        
# save the number of blocks
Q_early_adult = max(early_adult_membership_list)
        
# add the block memberships to the phenotype dataframe
early_adult_phenotype_df['Block'] = early_adult_membership_list

In [ ]:
# Permutation test for the youth age group
group1_vals = youth_phenotype_df[youth_phenotype_df['Block'] == 1]['SIP'].values
group2_vals = youth_phenotype_df[youth_phenotype_df['Block'] == 2]['SIP'].values

res = permutation_test((group1_vals, group2_vals),
                       statistic=lambda x, y: np.mean(x) - np.mean(y),
                       permutation_type='independent',
                       alternative='two-sided',
                       n_resamples=10000,
                       random_state=1234)

print("Permutation test for between-block comparison of mean SIP scores among the youth age group")
print(f"Observed difference: {res.statistic:.3f}, p-value: {res.pvalue:.4f}\n")

In [ ]:
# Permutation tests for the early adult age group
group1_vals = early_adult_phenotype_df[early_adult_phenotype_df['Block'] == 1]['SIP'].values
group2_vals = early_adult_phenotype_df[early_adult_phenotype_df['Block'] == 2]['SIP'].values

res = permutation_test((group1_vals, group2_vals),
                       statistic=lambda x, y: np.mean(x) - np.mean(y),
                       permutation_type='independent',
                       alternative='two-sided',
                       n_resamples=10000,
                       random_state=1234)

print("Permutation test for between-block comparison of mean SIP scores among the early adult age group")
print(f"Observed difference: {res.statistic:.3f}, p-value: {res.pvalue:.4f}\n")